Based on these plots, the search criteria should be:
* $|l| < 15^\circ$
* $-20^\circ < b < 40^\circ$
* G > 10
* $\mu_b < -10~{\rm mas}~{\rm yr}^{-1}$

In [ ]:
import os

from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
import h5py

from gala.observation import apparent_magnitude
import gala.dynamics as gd
import gala.coordinates as gc
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('apw-notebook')
plt.rcParams['figure.dpi'] = 72
%matplotlib notebook

In [ ]:
tgas_path = os.path.abspath("../data/tgas.hdf5")

In [ ]:
d = np.load("../data/best-ophiuchus-model.npy")

In [ ]:
oph = gd.CartesianPhaseSpacePosition(pos=d[:3]*u.kpc, 
                                     vel=d[3:]*u.kpc/u.Myr)

In [ ]:
fig,ax = plt.subplots(figsize=(10,10), subplot_kw=dict(projection='3d'))
ax.plot(*oph.pos, linestyle='none')
ax.set_xlim(-15,15)
ax.set_ylim(-15,15)
ax.set_zlim(-15,15)

In [ ]:
idx = ((oph.pos[0] < 0*u.kpc) & (oph.pos[0] > -6*u.kpc) &
       (oph.pos[1] > -1*u.kpc) & (oph.pos[1] < 1*u.kpc) &
       (oph.pos[2] > -5*u.kpc) & (oph.pos[1] < 5*u.kpc))
       
oph_box = oph[idx]

In [ ]:
fig,ax = plt.subplots(figsize=(8,8), subplot_kw=dict(projection='3d'))
ax.plot(*oph_box.pos, linestyle='none')
ax.set_xlim(-15,15)
ax.set_ylim(-15,15)
ax.set_zlim(-15,15)

In [ ]:
gal,gal_v = oph_box.to_frame(coord.Galactic)

In [ ]:
plt.figure()
plt.plot(gal_v[0].to(u.mas/u.yr), gal_v[1].to(u.mas/u.yr), linestyle='none')
# plt.xlim(-60,60)
plt.xlabel('$\mu_l$ [deg]')
plt.ylabel('$\mu_b$ [deg]')

In [ ]:
plt.figure()
plt.plot(gal.b.to(u.degree), gal_v[0].to(u.mas/u.yr), linestyle='none')
plt.xlim(-60,60)
plt.xlabel('$b$ [deg]')
plt.ylabel('$\mu_l$ [deg]')

In [ ]:
plt.figure()
plt.plot(gal.b.to(u.degree), gal_v[1].to(u.mas/u.yr), linestyle='none')
plt.xlabel('$b$ [deg]')
plt.ylabel(r'$\mu_b$ [mas/yr]')
plt.xlim(-60,60)

In [ ]:
plt.figure()
plt.plot(gal.b.to(u.degree), gal.distance.to(u.kpc), linestyle='none')
plt.xlabel('$b$ [deg]')
plt.ylabel(r'$d_\odot$ [kpc]')
plt.xlim(-60,60)
plt.ylim(0, 10)

In [ ]:
plt.figure()
plt.plot(gal.b.to(u.degree), apparent_magnitude(0, gal.distance.to(u.kpc)), 
         linestyle='none', label='BHB', alpha=0.5)
plt.plot(gal.b.to(u.degree), apparent_magnitude(-2, gal.distance.to(u.kpc)), 
         linestyle='none', label='KG hi', alpha=0.5)
plt.plot(gal.b.to(u.degree), apparent_magnitude(0.5, gal.distance.to(u.kpc)), 
         linestyle='none', label='KG lo', alpha=0.5)

plt.xlabel('$b$ [deg]')
plt.ylabel(r'$m$ [mag]')

plt.xlim(-60,60)
plt.ylim(10, 15)

plt.axhline(12.5, linestyle='dashed')

plt.legend()

In [ ]:
icrs = gal.transform_to(coord.ICRS)

In [ ]:
plt.figure()
plt.plot(icrs.ra.to(u.degree), icrs.dec.to(u.degree),
         linestyle='none', alpha=0.5)
plt.xlim(360, 0)
plt.ylim(-60,60)

## Load Gaia!

In [ ]:
with h5py.File(tgas_path, 'r') as f:
    tgas_c = coord.SkyCoord(ra=f['tgas']['ra']*u.degree, dec=f['tgas']['dec']*u.degree, 
                            distance=(f['tgas']['parallax']*u.mas).to(u.kpc, equivalencies=u.parallax()))
    Gmag = f['tgas']['phot_g_mean_mag']
    parallax = f['tgas']['parallax']*u.mas
    
    pm = np.vstack((f['tgas']['pmra'], f['tgas']['pmdec']))*u.mas/u.yr
    
    parallax_err = f['tgas']['parallax_error']*u.mas
    pm_err = np.vstack((f['tgas']['pmra_error'], f['tgas']['pmdec_error']))*u.mas/u.yr

In [ ]:
pml,pmb = gc.pm_icrs_to_gal(tgas_c, pm)

In [ ]:
tgas_gal = tgas_c.galactic

In [ ]:
pm_SN = np.sqrt(np.sum(pm**2, axis=0) / np.sum(pm_err**2, axis=0))

In [ ]:
idx = ((np.abs(tgas_gal.l) < 10*u.deg) & 
       (tgas_gal.b > -40*u.deg) & (tgas_gal.b < 90*u.deg) &
       (Gmag > 10) &
       (pml < 0*u.mas/u.yr) & (pmb > 0*u.mas/u.yr) & 
       (pm_SN > 10) & 
       (parallax.to(u.kpc, equivalencies=u.parallax()) > 1.*u.kpc))

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(10,5))
axes[0].hist(parallax[idx], bins=np.linspace(0, 25, 32))
axes[0].set_yscale('log')

axes[1].hist(1/parallax[idx], bins=np.linspace(0, 10, 32))
axes[1].set_yscale('log')

In [ ]:
bins = (np.linspace(-25,0,64), np.linspace(0,25,64))
H,xedg,yedg = np.histogram2d(gal_v[0].to(u.mas/u.yr), gal_v[1].to(u.mas/u.yr), bins=bins)

In [ ]:
plt.figure()
plt.plot(pml.to(u.mas/u.yr).value[idx], # & (1/parallax > 2)], 
         pmb.to(u.mas/u.yr).value[idx],# & (1/parallax > 2)], 
         linestyle='none', alpha=0.75, marker='.', markersize=5, color='k')
plt.contourf(np.log10(H).T, extent=[-25,0,0,25], cmap='Blues')

plt.xlabel(r'$\mu_l$ [mas/yr]')
plt.ylabel(r'$\mu_b$ [mas/yr]')
plt.xlim(-25,0)
plt.ylim(0,25)

plt.tight_layout()